In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from keras import optimizers
from keras import models
from keras import layers
import numpy as np 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.models import Sequential, Model
from PIL import Image
import matplotlib.pyplot as plt
import glob
import skimage.io as io
import skimage.transform as trans

Using TensorFlow backend.


In [13]:
from data import *

In [29]:
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6],axis=3) 
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7],axis=3) 
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8],axis=3) 
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9],axis=3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [30]:
model = unet()
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next())
def myGenerator(train_generator,train_generator1):
    while True:
        xy = train_generator.next() #or next(train_generator)
        xy1 = train_generator1.next() #or next(train_generator1)
        yield (xy[0],xy1[0])

/home/lixing/.local/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [31]:
Skin = [0,0,0]
Unlabelled = [0,0,0]

COLOR_DICT = np.array([Skin, Unlabelled])

def adjustData(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255.0
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            #for one pixel in the image, find the class in mask and convert it into one-hot vector
            #index = np.where(mask == i)
            #index_mask = (index[0],index[1],index[2],np.zeros(len(index[0]),dtype = np.int64) + i) if (len(mask.shape) == 4) else (index[0],index[1],np.zeros(len(index[0]),dtype = np.int64) + i)
            #new_mask[index_mask] = 1
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255.0
        mask = mask /255.0
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(batch_size,aug_dict,image_color_mode = "grayscale",mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"

    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/train/image',
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = '/home/lixing/git/A1/data/train/aug',
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/train/label',
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = '/home/lixing/git/A1/data/train/aug_label',
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)
        yield (img,mask)
def labelVisualize(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255.0

def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.png"%i),img)        

In [32]:
data_gen_args = dict(rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,data_gen_args)
model = unet()
model_checkpoint = ModelCheckpoint('unet_skin_detection.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=1,epochs=10,callbacks=[model_checkpoint])        

/home/lixing/.local/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/10
Found 36 images belonging to 1 classes.
Found 36 images belonging to 1 classes.
1/1 [==============================] - 34s 34s/step - loss: 0.7088 - acc: 0.0680

Epoch 00001: loss improved from inf to 0.70882, saving model to unet_skin_detection.hdf5
Epoch 2/10
1/1 [==============================] - 13s 13s/step - loss: 0.6701 - acc: 0.9443

Epoch 00002: loss improved from 0.70882 to 0.67007, saving model to unet_skin_detection.hdf5
Epoch 3/10
1/1 [==============================] - 8s 8s/step - loss: 0.4638 - acc: 0.9706

Epoch 00003: loss improved from 0.67007 to 0.46378, saving model to unet_skin_detection.hdf5
Epoch 4/10
1/1 [==============================] - 8s 8s/step - loss: 0.2662 - acc: 0.9684

Epoch 00004: loss improved from 0.46378 to 0.26621, saving model to unet_skin_detection.hdf5
Epoch 5/10
1/1 [==============================] - 8s 8s/step - loss: 0.3344 - acc: 0.9669

Epoch 00005: loss did not improve from 0.26621
Epoch 6/10
1/1 [==============================

In [33]:
test_datagen = ImageDataGenerator(rescale=1./255.0)
test_generator = test_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/test',
        target_size=(256,256),
        batch_size=1,
        color_mode="grayscale",
        class_mode='binary')
def testGenerator(test_generator):
    while True:
        xy = test_generator.next() #or next(train_generator)
        yield (xy[0])
test_image = testGenerator(test_generator)

Found 43 images belonging to 1 classes.


In [35]:
model_test = unet()
model_test.load_weights("/home/lixing/git/A1/unet_skin_detection.hdf5")
results = model.predict_generator(test_image,10,verbose=1)
saveResult("/home/lixing/git/A1/data/test",results)

/home/lixing/.local/lib/python3.5/site-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


10/10 [==============================] - 11s 1s/step


In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255.0,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
seed= 1

train_images_generator = train_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/train/image',
        target_size=(256,256),
        batch_size=1,
        color_mode="rgb",
        class_mode='binary',
        save_to_dir='/home/lixing/git/A1/data/train/aug', 
        save_format='png',
        seed=seed)
train_masks_generator = train_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/train/label',
        target_size=(256,256),
        batch_size=1,
        color_mode="grayscale",
        class_mode='binary',
        save_to_dir='/home/lixing/git/A1/data/train/aug_label', 
        save_format='png',
        seed=seed)

validation_images_generator = train_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/validation/image_validation',
        target_size=(256,256),
        batch_size=1,
        color_mode="rgb",
        class_mode='binary',
        seed=seed)

validation_masks_generator = train_datagen.flow_from_directory(
        '/home/lixing/git/A1/data/validation/label_validation',
        target_size=(256,256),
        batch_size=1,
        color_mode="grayscale",
        class_mode='binary',
        seed=seed)

        
combinedGenerator = myGenerator(train_images_generator, train_masks_generator)
validation_generator = myGenerator(validation_images_generator,validation_masks_generator)
history = model.fit_generator(
        combinedGenerator,
        steps_per_epoch=1,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=50)